In [1]:
import numpy as np
from mflex.model.field.utility.poloidal import (phi_hypgeo, phi, dphidz, dphidz_hypgeo)

In [2]:
xmin = 0.0
ymin = 0.0
zmin = 0.0
xmax = 2.0
ymax = 2.0
zmax = 2.0

nresol_x = 400
nresol_y = 400
nresol_z = 400
nf_max = 400

pixelsize_x = (xmax - xmin) / nresol_x
pixelsize_y = (ymax - ymin) / nresol_y
pixelsize_z = (zmax - zmin) / nresol_z

z0 = 0.2
b = 1.0

length_scale = 2.0
length_scale_x = 1.0 * nresol_x * float(pixelsize_x)
length_scale_y = 1.0 * nresol_y * float(pixelsize_y)
length_scale_x_norm = length_scale_x / length_scale
length_scale_y_norm = length_scale_y / length_scale
x_arr = np.arange(nresol_x) * (xmax - xmin) / (nresol_x - 1) + xmin
y_arr = np.arange(nresol_y) * (ymax - ymin) / (nresol_y - 1) + ymin
z_arr = np.arange(nresol_z) * (zmax - zmin) / (nresol_z - 1) + zmin

kx_arr = np.arange(nf_max) * np.pi / length_scale_x_norm  # [0:nf_max]
ky_arr = np.arange(nf_max) * np.pi / length_scale_y_norm  # [0:nf_max]

one_arr = 0.0 * np.arange(nf_max) + 1.0
ky_grid = np.outer(ky_arr, one_arr)  # [0:nf_max, 0:nf_max]
kx_grid = np.outer(one_arr, kx_arr)  # [0:nf_max, 0:nf_max]
k2_arr = np.outer(ky_arr**2, one_arr) + np.outer(one_arr, kx_arr**2)
k2_arr[0, 0] = (np.pi / length_scale) ** 2 + (np.pi / length_scale) ** 2

deltaz = z0 / 10.0


In [3]:
a = 0.22
alpha = 0.25
ratiodzls = deltaz / length_scale

In [4]:
p_arr = 0.5 * ratiodzls * np.sqrt(k2_arr * (1.0 - a - a * b) - alpha**2)
q_arr = 0.5 * ratiodzls * np.sqrt(k2_arr * (1.0 - a + a * b) - alpha**2)

In [5]:
phi_asymp = np.zeros((nf_max, nf_max, nresol_z))
phi_hyp = np.zeros((nf_max, nf_max, nresol_z))
dphi_asymp = np.zeros((nf_max, nf_max, nresol_z))
dphi_hyp = np.zeros((nf_max, nf_max, nresol_z))

In [6]:
def check_runtime_pa():
    for iz, z in enumerate(z_arr):
        phi_asymp[:, :, iz] = phi(z, p_arr, q_arr, z0, deltaz)

In [7]:
def check_runtime_ph():
    for iz, z in enumerate(z_arr):
        phi_hyp[:, :, iz] = phi_hypgeo(z, p_arr, q_arr, z0, deltaz)

In [8]:
def check_runtime_dpa():
    for iz, z in enumerate(z_arr):
        dphi_asymp[:, :, iz] = dphidz(z, p_arr, q_arr, z0, deltaz)

In [9]:
def check_runtime_dph():
    for iz, z in enumerate(z_arr):
        dphi_hyp[:, :, iz] = dphidz_hypgeo(z, p_arr, q_arr, z0, deltaz)

In [10]:
%timeit -r 1000 check_runtime_pa()

1.17 s ± 29.3 ms per loop (mean ± std. dev. of 1000 runs, 1 loop each)


In [11]:
%timeit -r 1000 check_runtime_ph()

KeyboardInterrupt: 

In [ ]:
%timeit -r 1000 check_runtime_dpa()

159 ms ± 4.89 ms per loop (mean ± std. dev. of 1000 runs, 1 loop each)


In [ ]:
%timeit -r 1000 check_runtime_dph()

3.84 s ± 93.3 ms per loop (mean ± std. dev. of 1000 runs, 1 loop each)
